# Day047
## 超參數調整與優化
超參數(Hyper Parameter)會影響模型的結果，但提昇效果有限，最重要的還是資料清理與特徵工程，建議先由預設值慢慢調整
### 超參數調整方法
1. 窮舉法(Grid Search): 直接指定超參數的組合範圍，每一組參數都訓練完成後再根據驗證集 (validation) 的結果選擇最佳參數
2. 隨機搜尋(Random Search): 指定超參數的範圍，用均勻分布進行參數抽樣，用抽到的參數進行訓練，再根據驗證集的結果選擇最佳參數，通常會比Grid Search還好，[Smarter Parameter Sweeps (or Why Grid Search Is Plain Stupid)](https://medium.com/rants-on-machine-learning/smarter-parameter-sweeps-or-why-grid-search-is-plain-stupid-c17d97a0e881)


### 正確的超參數調整步驟
- 若持續使用同一份驗證集 (validation) 來調參，可能讓模型的參數過於擬合該驗證集，正確的步驟是使用 Cross-validation 確保模型泛化性
- 先將資料切分為訓練/測試集，測試集保留不使用•將剛切分好的訓練集，再使用Cross-validation 切分 K 份訓練/驗證集
- 用 grid/random search 的超參數進行訓練與評估•選出最佳的參數，用該參數與全部訓練集建模- - 最後使用測試集評估結果


# 範例

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取手寫辨識資料集
boston = datasets.load_boston()
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 8.379 的 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

8.913775994322064


In [4]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\ChihYing\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    6.3s finished
C:\Users\ChihYing\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -13.118704 using {'max_depth': 3, 'n_estimators': 200}


In [6]:
grid_result.best_params_

{'max_depth': 3, 'n_estimators': 200}

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [8]:
# 調整參數後約可降至 8.30 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

8.544204634077806


# 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合
使用[RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

iris = datasets.load_iris()
train_X, test_X, train_Y, test_Y = train_test_split(iris.data, iris.target, test_size=0.25, random_state=4)
clf = GradientBoostingClassifier()
clf.fit(train_X, train_Y)
pred = clf.predict(test_X)
print(metrics.accuracy_score(test_Y, pred))

0.9736842105263158


In [10]:
param_dist = {"max_depth": [1, 3, 5, 7],
              "min_samples_split": list(range(2,100)),
              "n_estimators": list(range(1,100)),
              "learning_rate": [0.05, 0.1]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, random_state=4)

random_result = random_search.fit(train_X, train_Y)
print("Best Accuracy: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best Accuracy: 0.973214 using {'n_estimators': 46, 'min_samples_split': 37, 'max_depth': 7, 'learning_rate': 0.05}


C:\Users\ChihYing\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [19]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(n_estimators=46, min_samples_split=37, max_depth=7, learning_rate=0.05)

# 訓練模型
clf_bestparam.fit(train_X, train_Y)

# 預測測試集
y_pred = clf_bestparam.predict(test_X)

print(metrics.accuracy_score(test_Y, y_pred))

0.9736842105263158
